In [30]:
import pandas as pd
from tqdm.auto import tqdm

In [2]:
data = pd.read_csv("extracted_data.csv")

In [3]:
data

,Date,link,text
0,"October 04, 2023 11:16 AM IST",https://www.moneycontrol.com/news/business/mar...,HDFC Bank's stock has declined by over 9 perce...
1,"October 03, 2023 04:14 PM IST",https://www.moneycontrol.com/news/business/hdf...,HDFC Bank Shares Have Trailed Private Lenders ...
2,"September 24, 2023 11:40 AM IST",https://www.moneycontrol.com/news/business/ban...,The combined market valuation of eight of the ...
3,"September 21, 2023 04:34 PM IST",https://www.moneycontrol.com/news/business/hdf...,HDFC AMC holds a 4.49 per cent stake in Kerala...
4,"August 25, 2023 06:54 PM IST",https://www.moneycontrol.com/news/business/ban...,HDFC Bank\n\nlive bse live\n\nnse live Volume ...
...,...,...,...
744,"December 05, 2016 09:50 AM IST",https://www.moneycontrol.com/news/business/mar...,Moneycontrol Bureau9:55 am Deal seal? Tata Ste...
745,"December 02, 2016 11:10 AM IST",https://www.moneycontrol.com/news/business/mar...,Moneycontrol Bureau10:59 am Market Update: The...
746,"November 25, 2016 10:22 PM IST",https://www.moneycontrol.com/news/business/mar...,Moneycontrol Bureau\n\nSpectacular rally on Fr...
747,"November 25, 2016 11:03 AM IST",https://www.moneycontrol.com/news/business/mar...,Moneycontrol Bureau9:55 am Market check: The S...


In [7]:
data["Date"] = pd.to_datetime(data["Date"])

/tmp/ipykernel_102132/2487158108.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"])
/tmp/ipykernel_102132/2487158108.py:1: FutureWarning: Parsing '{res.tzname}' as tzlocal (dependent on system timezone) is deprecated and will raise in a future version. Pass the 'tz' keyword or call tz_localize after construction instead
  data["Date"] = pd.to_datetime(data["Date"])


In [8]:
data["len"] =  data["text"].apply(lambda x: len(x.split()))

In [9]:
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
model_name = "yiyanghkust/finbert-tone"

In [12]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis", model=model_name)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [21]:
pd.DataFrame(nlp(["I am very sad"]))["label"].map(MAP)

0    1
Name: label, dtype: int64

In [20]:
MAP = {
    "Positive": 1,
    "Negative": -1,
    "neutral": 0
}

In [23]:
def sentiment_article(article):
    pieces = article.split("\n")
    analysis = nlp(pieces)
    analysis = pd.DataFrame(analysis)
    analysis["factor"] = analysis["label"].map(MAP)
    analysis["new_score"] = analysis["score"]*analysis["factor"]
    return analysis["new_score"].mean()

In [36]:
sentiment = []
for _, row in tqdm(data.iterrows(), total=len(data)):
    try:
        sentiment.append(sentiment_article(row["text"]))
    except:
        sentiment.append(0)

  0%|          | 0/749 [00:00<?, ?it/s]

In [37]:
data["sentiment"] = sentiment

In [ ]:
data.to_csv("final_data.csv", index=False)